# Librerías

# Cargar Dataset

# Prueba Chat

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch_geometric.data import DataLoader
from dataset_utils import generate_edge_index, generate_edge_attr, build_graph_list, build_graph_from_sample, load_model_by_name
from train_eval import evaluate, evaluate_NNConv
from gcn_model import GCN
from gat_model import GAT
from sage_model import GraphSAGE
from nnconv_model import NNConvNet

In [ ]:
dir_path = os.getcwd()
print(dir_path)
folder_base = "saved_models"
print(folder_base)
architectures = ["GCN", "GAT", "SAGE", "NNConv"]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Configuración del modelo
config = {
    "input_dim": 3,           # 3 canales: T_interfaces, Q_heaters, T_env
    "edge_dim": 3,           # Dimensión de los atributos de las aristas (GR, GLx, GLy)
    "hidden_dim": 128,
    "output_dim": 1,
    "num_layers": 3,
    "attention_heads": 4,
    "use_dropout": True,
    "dropout_rate": 0.15217808088248522,
    "use_batchnorm": True,
    "use_residual": True,
    "batch_size": 16,
    "lr": 0.0022165075808203588,
    "max_epochs": 10000,
    "early_stop_patience": 100,
    "error_threshold_K": 3.0,
    "use_percentage_threshold": False,
    "percentage_threshold": 1.0,  # solo si se activa la opción
    "save_model": True,
    "load_model": False, # Cargar un modelo preentrenado
    "factor" : 0.5 , # Factor de reducción del learning rate
    "patience" : 10 , # Épocas sin mejora para reducir el learning rate
    "min_lr" : 1e-9,  # evita que baje infinitamente el learning rate
    "use_physics" : True,
    "lambda_physics" : 0.001, # Peso de la física en la función de pérdida
    "use_boundary_loss" : True,
    "lambda_boundary" : 1,
    "use_heater_loss" : True,
    "lambda_heater" : 10
}

In [ ]:

norm_dataset_path = os.path.join(dir_path, "Datasets", "PCB_Dataset_Norm.pth")
info_path = os.path.join(dir_path, "Datasets", "normalization_info.pth")

norm_dataset = torch.load(norm_dataset_path)
norm_info = torch.load(info_path)

grid_size = norm_dataset.inputs.shape[-1]
print("PCB cuadrada de", grid_size, "nodos por lado")

edge_index = generate_edge_index(grid_size=grid_size)
edge_attr = generate_edge_attr(grid_size=grid_size, edge_index=edge_index)
graphs = build_graph_list(norm_dataset, edge_index, edge_attr)


test_loader = DataLoader(norm_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def get_model_class(arch):
    if arch == "GCN": return GCN
    if arch == "GAT": return GAT
    if arch == "SAGE": return SAGE
    if arch == "NNConv": return NNConvNet
    raise ValueError(f"Arquitectura no reconocida: {arch}")


def get_model_config_from_filename(filename):
    # Añade aquí si quieres parsear hiperparámetros del nombre
    return {
        "input_dim": 3,
        "hidden_dim": 128,
        "output_dim": 1,
        "num_layers": 3,
        "use_dropout": True,
        "dropout_rate": 0.2,
        "use_batchnorm": False,
        "use_residual": False,
        "edge_dim": 3  # solo para NNConv
        "attention_heads": 4,  # solo para GAT
    }

In [ ]:
resultados = {}

for arch in architectures:
    model_dir = os.path.join(folder_base, arch)
    if not os.path.isdir(model_dir):
        print(f"No se encontró la carpeta de modelos para {arch}.")
        continue

    model_files = [f for f in os.listdir(model_dir) if f.endswith(".pth")]
    if not model_files:
        print(f"No hay modelos en {model_dir}")
        continue

    # Se puede elegir uno específico o el último por ejemplo
    filename = sorted(model_files)[-1]
    model_config = get_model_config_from_filename(filename)
    ModelClass = get_model_class(arch)

    if arch == "NNConv":
        model = ModelClass(
            input_dim=model_config["input_dim"],
            hidden_dim=model_config["hidden_dim"],
            output_dim=model_config["output_dim"],
            num_layers=model_config["num_layers"],
            edge_dim=model_config["edge_dim"],
            use_dropout=model_config["use_dropout"],
            dropout_rate=model_config["dropout_rate"],
            use_batchnorm=model_config["use_batchnorm"],
            use_residual=model_config["use_residual"]
        ).to(device)
    if    arch == "GAT":
        model = ModelClass(
            input_dim=model_config["input_dim"],
            hidden_dim=model_config["hidden_dim"],
            output_dim=model_config["output_dim"],
            num_layers=model_config["num_layers"],
            attention_heads=model_config["attention_heads"],
            use_dropout=model_config["use_dropout"],
            dropout_rate=model_config["dropout_rate"],
            use_batchnorm=model_config["use_batchnorm"],
            use_residual=model_config["use_residual"]
        ).to(device)
    else:
        model = ModelClass(
            input_dim=model_config["input_dim"],
            hidden_dim=model_config["hidden_dim"],
            output_dim=model_config["output_dim"],
            num_layers=model_config["num_layers"],
            use_dropout=model_config["use_dropout"],
            dropout_rate=model_config["dropout_rate"],
            use_batchnorm=model_config["use_batchnorm"],
            use_residual=model_config["use_residual"]
        ).to(device)

    model_path = os.path.join(model_dir, filename)
    model = load_model_by_name(model, filename, folder=model_dir)

    print(f"\nEvaluando {arch} desde archivo: {filename}")
    plot = True  # Solo el último modelo graficado

    results = evaluate(
        model, test_loader, device, norm_info,
        lambda_physics=0.0,
        use_physics=False,
        lambda_boundary=0.0,
        use_boundary_loss=False,
        lambda_heater=0.0,
        use_heater_loss=False,
        error_threshold=2.0,
        percentage_threshold=None,
        plot_results=plot
    )

    resultados[arch] = results

In [ ]:
# --- CELDA 6: Comparación visual o resumen ---
metricas = ["MSE", "MAE", "R2", "Accuracy"]

valores = {m: [] for m in metricas}

for arch in architectures:
    if arch not in resultados:
        for m in metricas:
            valores[m].append(None)
        continue

    mse, mae, r2, acc = resultados[arch][0], resultados[arch][1], resultados[arch][2], resultados[arch][3]
    valores["MSE"].append(mse)
    valores["MAE"].append(mae)
    valores["R2"].append(r2)
    valores["Accuracy"].append(acc)

x = np.arange(len(architectures))
width = 0.2

fig, ax = plt.subplots(figsize=(10, 6))

for i, m in enumerate(metricas):
    ax.bar(x + i * width, valores[m], width, label=m)

ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(architectures)
ax.set_ylabel("Valor")
ax.set_title("Comparación de métricas por arquitectura")
ax.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()